## Функции для работы с данными

In [0]:
## функция для просмотра пропусков в данных

def values_table_null(data):
        # всего пропусков
        mis_val = data.isnull().sum()
        
        # процент пропусков
        mis_val_percent = 100 * data.isnull().sum() / len(data)
        
        # Делаем таблицу с получившимися результатами
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Переименовать столбцы
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Сортировка таблицы по проценту пропущенных по убыванию
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Выввод на печать сводную информацию
        print ("Выбранныей Вами данные " + str(data.shape[1]) + " колонок.\n"      
            "Из них " + str(mis_val_table_ren_columns.shape[0]) +
              " имеют пропуск")      

        return mis_val_table_ren_columns

## Функция по алгоритмам с метрикой "accurary"

In [0]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV


def fit_model(model, X, y, parameters):  
  cross_validation = StratifiedKFold(n_splits=5)

  grid_search = GridSearchCV(model,
                              scoring='accuracy',
                              param_grid=parameters,
                              cv=cross_validation,
                              verbose=1
                            )

  grid_search.fit(X, y)
  parameters=grid_search.best_params_
  print('Best score: {}'.format(grid_search.best_score_))
  print('Best parameters: {}'.format(parameters))

  return grid_search

## Чистка данных (удаление колонок, которые имеют пропуски более 50 %)

In [0]:
missing_df = values_table_null(df);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('Удаляем %d колонок.' % len(missing_columns))

In [0]:
# Нормировка, нормализация и стандартизация
for col in float_cols:
  df[col] = df[col].apply(lambda x: x/df[col].mean())
  
yeo = PowerTransformer(method='yeo-johnson', standardize=False)
for col in float_cols:
    df[col] = yeo.fit_transform(df[col].values.reshape(df.shape[0],-1))
    
scaler = StandardScaler()
scaler.fit(df[float_cols])
df[float_cols] = scaler.transform(df[float_cols])

In [0]:
# поиск лучших фич и удаляем лишнее
def best_features(X, y):
  #выбираем топ 10 фич
  bestfeatures = SelectKBest(score_func=chi2, k=10)
  fit = bestfeatures.fit(X,y)
  dfscores = pd.DataFrame(fit.scores_)
  dfcolumns = pd.DataFrame(X.columns)
  featureScores = pd.concat([dfcolumns,dfscores],axis=1)
  featureScores.columns = ['Specs','Score']  
  return list(featureScores.nlargest(10,'Score')['Specs'])   

In [0]:
# стандартизируем
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(x)